In [1]:
import pandas as pd
import numpy as np

In [2]:
class schedule_tuple:
    def __init__(self, schedule_pairs):
        self.schedule_pairs = schedule_pairs
        self.roads = [i[0] for i in self.schedule_pairs]
        self.time = [int(i[1]) for i in self.schedule_pairs]
        self.cumulated_time = np.cumsum(self.time)
        self.cycle = self.cumulated_time[-1]
        self.schedule_queue = [] #to depleted next time
        for r,t in self.schedule_pairs:
            self.schedule_queue.extend([r] * int(t))
    def get(self, t):
        t = t % self.cycle
        return self.schedule_queue[t]
        
class traffic_light:
    def __init__(self, traffic_light_id, schedule_pairs):
        self.id = traffic_light_id # traffic light id
        self.schedule_pairs =  schedule_pairs#traffic light schedule, output of the simulation, in the form of pairs [(road1, 4),(road2, 4)]
        self.schedule_tuple = schedule_tuple(schedule_pairs) #traffic light queue marking green light road at each time, not mutable
    def get_current_road_w_green_light(self, t):
        return self.schedule_tuple.get(t)
    
class road:
    def __init__(self, road_id, travel_time):
        self.id = road_id # road id
        self.travel_time = travel_time
        self.waiting_queue = [] #car queue waiting at the end
        self.car_tracker = {} # car travelling (not waiting), distance_to_end pair
    
    def time_elapse_and_return_finished_cars(self):
        gone_cars = []
        f_cars = []
        for car, length_left in self.car_tracker.items():
            if length_left == 1:
                if car.on_final_path:
                    f_cars.append(car)
                else:
                    self.waiting_queue.append(car)
                gone_cars.append(car)
            else:
                self.car_tracker[car] = length_left - 1
        for car in gone_cars:
            self.car_tracker.pop(car)
        return f_cars
    
    def get_first_car(self):
        if self.waiting_queue == []:
            return None
        else:
            first_car = self.waiting_queue[0]
            self.waiting_queue = self.waiting_queue[1:]
        return first_car
    
    def car_enter(self, car_object):
        self.car_tracker[car_object] = self.travel_time
        
class car:
    def __init__(self, car_id, path_list):
        self.id = car_id # car id
        self.path_list = path_list # entire path ordered, each element is the road name
        self.current_path_number = 0 # serial number of current path, say 6 means travelling on / waiting at the end of the 6th path in path_list
        self.num_paths = len(self.path_list)
        self.on_final_path = False
    
    def make_crossing(self):
        next_path = self.get_next_path()
        next_path.car_enter(self)
        self.current_path_number += 1
        if self.current_path_number == self.num_paths - 1:
            self.on_final_path = True

    def init_on_road(self):
        if self.current_path_number == self.num_paths - 1:
            self.on_final_path = True
        return self.path_list[0].car_enter(self)
    
    def get_next_path(self):
        return self.path_list[self.current_path_number + 1]

In [3]:
with open("hashcode.in") as f:
    D, I, S, V, F = tuple(int(temp) for temp in f.readline().split())
    street_info = {}
    for temp in range(S):
        street_info[temp]=tuple(temp for temp in f.readline().split())
    car_info = {}
    for temp in range(V):
        car_info[temp]=tuple(temp for temp in f.readline().split())

In [4]:
street_info = pd.DataFrame.from_dict(street_info, orient = 'index',columns = ['start_intersection','end_intersection','street_name','travel_time']).set_index('street_name')
for c in street_info.columns:
    street_info[c] = street_info[c].astype(int)
street_info['road_object'] = street_info.apply(lambda row: road(row.name, row.travel_time), axis = 1)
street_info['end_light_schedule'] = np.ones(street_info.shape[0],).astype('int')

In [5]:
car_info = pd.DataFrame.from_dict(car_info, orient = 'index',columns = ['n_streets'] + ['street_'+ str(i) for i in range(120)])
car_info['path'] = car_info.apply(lambda x: [street_info['road_object'][i] for i in x[1:121]], axis = 1)
car_info['car_object'] = car_info.apply(lambda row: car(row.name, row.path), axis = 1) #take only 1 road path for testing

In [6]:
traffic_light_info = pd.DataFrame(street_info.groupby('end_intersection').apply(lambda x: list(x.road_object)), columns = ['enter_from'])
traffic_light_info['exit_to'] = street_info.groupby('start_intersection').apply(lambda x: list(x.road_object))
traffic_light_info['schedule'] = traffic_light_info['enter_from'].apply(lambda lst: [1 for i in lst])
traffic_light_info['schedule_pairs'] = traffic_light_info.apply(lambda row: list(zip(row['enter_from'],row['schedule'])), axis = 1)
traffic_light_info['traffic_light_object'] = traffic_light_info.apply(lambda row: traffic_light(row.name, row.schedule_pairs), axis = 1)
#traffic_light_info.head()

In [7]:
# pseudu code
def light_operating(light):
    road_w_green_light = light.get_current_road_w_green_light(t)
    crossing_car = road_w_green_light.get_first_car() #find the car object
    if crossing_car is None:
        pass
    else:
        crossing_car.make_crossing()
t = 0 
cars_on_the_road = list(car_info.car_object)#all cars
score = 0
car_info.car_object.apply(lambda car: car.init_on_road())
finished_cars = []
print('t = ', t,'. number of cars on the road =', len(cars_on_the_road))
while cars_on_the_road != [] and t <= D:
    traffic_light_info.traffic_light_object.apply(light_operating)
    finished_cars.extend(street_info.road_object.apply(lambda road: road.time_elapse_and_return_finished_cars()))
    for finished_car in finished_cars:
        if not isinstance(finished_car, list):
            cars_on_the_road.remove(finished_car)
            score += F + D - t
    finished_cars = []
    
    t += 1
    if t%300 == 0:
        print('t = ', t,'. number of cars on the road =', len(cars_on_the_road))

t =  0 . number of cars on the road = 1000
t =  300
t =  300 . number of cars on the road = 1000
t =  600
t =  600 . number of cars on the road = 1000
t =  900
t =  900 . number of cars on the road = 1000
t =  1200
t =  1200 . number of cars on the road = 1000
t =  1500
t =  1500 . number of cars on the road = 1000
t =  1800
t =  1800 . number of cars on the road = 1000
t =  2100
t =  2100 . number of cars on the road = 1000
t =  2400
t =  2400 . number of cars on the road = 1000
t =  2700
t =  2700 . number of cars on the road = 1000
t =  3000
t =  3000 . number of cars on the road = 1000
t =  3300
t =  3300 . number of cars on the road = 1000
t =  3600
t =  3600 . number of cars on the road = 1000
t =  3900
t =  3900 . number of cars on the road = 1000
t =  4200
t =  4200 . number of cars on the road = 1000
t =  4500
t =  4500 . number of cars on the road = 1000
t =  4800
t =  4800 . number of cars on the road = 1000
t =  5100
t =  5100 . number of cars on the road = 1000
t =  5400
t

In [8]:
for i in cars_on_the_road:
    print(i.on_final_path)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [9]:
street_info.road_object[0]